In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import transformers
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer

In [1]:
pwd

'C:\\Users\\Karthiek Duggirala\\Downloads'

In [ ]:
# load dataframe
path = '/content/preprocessed_data.csv'
df = pd.read_csv(path)

In [ ]:
# Encode the genre labels
label_encoder = LabelEncoder()
df['genre_code'] = label_encoder.fit_transform(df['genre'])

In [ ]:
# Split the dataset into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(df['lyrics'], df['genre_code'], test_size=0.2, stratify = df['genre'],random_state=42)


In [ ]:
# Initialize TfidfVectorizer with a maximum of 10000 features
vectorizer = TfidfVectorizer(max_features=2000)

# Fit and transform the training data
train_vectors = vectorizer.fit_transform(train_data)

# Transform the testing data
test_vectors = vectorizer.transform(test_data)

In [ ]:
# Convert the numerical vectors into dense tensors
train_vectors = tf.convert_to_tensor(train_vectors.toarray(), dtype=tf.float32)
test_vectors = tf.convert_to_tensor(test_vectors.toarray(), dtype=tf.float32)

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_vectors, train_labels)).shuffle(len(train_data)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((test_vectors, test_labels)).batch(16)

In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(train_vectors.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
# Define the optimizer, loss function and evaluation metric
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [1]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
3227/3227 [==============================] - 13s 4ms/step - loss: 0.1271 - accuracy: 0.9618 - val_loss: 1.1236 - val_accuracy: 0.6867
Epoch 2/10
3227/3227 [==============================] - 13s 4ms/step - loss: 0.0688 - accuracy: 0.9824 - val_loss: 1.3198 - val_accuracy: 0.6867
Epoch 3/10
3227/3227 [==============================] - 13s 4ms/step - loss: 0.0462 - accuracy: 0.9882 - val_loss: 1.4287 - val_accuracy: 0.6885
Epoch 4/10
3227/3227 [==============================] - 14s 4ms/step - loss: 0.0358 - accuracy: 0.9909 - val_loss: 1.4891 - val_accuracy: 0.6877
Epoch 5/10
3227/3227 [==============================] - 14s 4ms/step - loss: 0.0334 - accuracy: 0.9968 - val_loss: 1.5980 - val_accuracy: 0.6899
Epoch 6/10
3227/3227 [==============================] - 14s 4ms/step - loss: 0.0358 - accuracy: 0.9972 - val_loss: 1.4464 - val_accuracy: 0.6872
Epoch 7/10
3227/3227 [==============================] - 14s 4ms/step - loss: 0.0322 - accuracy: 0.9958 - val_loss: 1.5213 - val_ac

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

807/807 [==============================] - 2s 2ms/step - loss: 1.4891 - accuracy: 0.6877
Test Loss: 1.489052653312683, Test Accuracy: 0.6877421140670776
